# Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('NLP_ass_train.tsv', sep='\t')
val = pd.read_csv('NLP_ass_valid.tsv', sep='\t')
test = pd.read_csv('NLP_ass_test.tsv', sep='\t')

train.columns = ['text', 'label']
val.columns = ['text', 'label']
test.columns = ['text', 'label']

In [ ]:
train

,text,label
0,nawt yall niggers ignoring me,normal
1,<user> i am bit confused coz chinese ppl can n...,hatespeech
2,and this is why i end up with nigger trainee d...,hatespeech
3,nogs jews and dykes how enriching,offensive
4,guilty until proven innocent unless you are a ...,hatespeech
...,...,...
15377,this vile racist ghetto ass dumber than a reta...,offensive
15378,when i first got on here and said i hate trump...,offensive
15379,was macht der moslem wenn der zion gegen seine...,normal
15380,it is awful look at world demographics asians ...,hatespeech


from matplotlib import pyplot as plt
import seaborn as sns
train.groupby('label').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
test

,text,label
0,laura loomer raped me while screaming at me in...,hatespeech
1,<user> what did the old lady do was she a nazi...,normal
2,as much as i appreciate a format of open discu...,normal
3,sex be so good a bitch be slow stroking and cr...,offensive
4,<user> <user> <user> with a room full of white...,normal
...,...,...
1918,it always women trying this shit like dingbat ...,offensive
1919,because women would never lie about being sexu...,offensive
1920,how is all that awesome muslim diversity going...,offensive
1921,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00


# Model Loading

In [ ]:
model_size = 'base'

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
import torch

model = AutoModelForSeq2SeqLM.from_pretrained(f"google/flan-t5-{model_size}")
tokenizer = AutoTokenizer.from_pretrained(f"google/flan-t5-{model_size}")
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

# Dataset Preparation

In [ ]:
from string import Template
from torch.utils.data import Dataset, DataLoader

label2id = {
    'normal': 0,
    'hatespeech': 1,
    'offensive': 2
}

id2label = {
    0: 'normal',
    1: 'hatespeech',
    2: 'offensive'
}

option2id = {
    'A': 0,
    'B': 1,
    'C': 2
}


class HateSpeechDataset(Dataset):
  def __init__(self, data, tokenizer, prompt_type, fewshot, preamble, ans_type, options, template, device):
    self.text = list(data['text'])
    self.labels = list(data['label'].apply(lambda x: label2id[x]))
    self.prompt_type = prompt_type
    self.fewshot = fewshot
    self.ans_type = ans_type
    self.preamble = preamble
    self.options = options
    self.template = template
    self.tokenizer = tokenizer
    self.device = device

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = self.text[idx]
    label = self.labels[idx]

    if self.prompt_type:
      input_text = self.template.substitute(
          preamble=self.preamble, fewshot=self.fewshot, prompt=text, options=self.options
      )

    else:
      input_text = self.template.substitute(
          preamble=self.preamble, prompt=text, options=self.options
      )

    # print(input_text)
    encoding = self.tokenizer(
            input_text,
            return_tensors='pt'
        ).to(self.device)

    return encoding, label

In [ ]:
prompt_type = 'zero_shot'
preamble = 'Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.'
options = ''
ans_type = 'label'
template = '$preamble\nSentence: $prompt\n$options\nAnswer:'
test_dataset = HateSpeechDataset(test, tokenizer, prompt_type, None, preamble, ans_type, options, Template(template), device)

In [ ]:
test_dataset[0]

Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.
Sentence: laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate

Answer:


({'input_ids': tensor([[   27,     7,     8,  7142,  1389,     6,  5591,     7,   855, 10217,
             42, 12130,    58,  4501,  4921,     8,  7142,   139,    80,    13,
            175,  1308,     3,    18,  1389,     6,  5591,     7,   855, 10217,
              6, 12130,     5,  4892,    17,  1433,    10,    50,  2414,     3,
          14351,    49,     3,    52,     9,  3138,   140,   298, 25471,    44,
            140,    16,   160, 27635,    53,     3,   157,  5208,  1612,    11,
            243,    62,   398,  1215,  6544,   342,     8,   281,    63,   603,
             50,  2414,     3, 14351,    49,     3, 14351,    49,  5339, 11801,
             10,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
templates = {}

In [ ]:
prompt_types = ['zero_shot', 'few_shot']

for prompt_type in prompt_types:
  templates[prompt_type] = {
      'small': [],
      'base' : []
  }

In [ ]:
def post_process(prediction, ans_type):
  if ans_type == 'label':
    if 'normal' in prediction.lower() or 'acceptable' in prediction.lower():
      return label2id['normal']
    elif 'hate' in prediction.lower():
      return label2id['hatespeech']
    elif 'offensive' in prediction.lower():
      return label2id['offensive']

  elif ans_type == 'option':
    if 'A' == prediction:
      return option2id['A']
    elif 'B' == prediction:
      return option2id['B']
    elif 'C' == prediction:
      return option2id['C']

# Zero-shot Inference

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm import tqdm

def inference(data):
  true_labels, pred_labels = [], []
  for i in tqdm(range(len(data))):
    input, label = data[i]
    output = model.generate(**input)
    pred = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # print(pred)
    pred_label = post_process(pred, data.ans_type)
    true_labels.append(label)
    pred_labels.append(pred_label)

  assert len(true_labels) == len(pred_labels)
  num_correct = sum(1 for a, b in zip(pred_labels, true_labels) if a == b)
  test_acc = num_correct / len(true_labels)
  macro_f1 = f1_score(true_labels, pred_labels, average='macro')

  print(classification_report(true_labels, pred_labels))
  print(confusion_matrix(true_labels, pred_labels))

  return test_acc, macro_f1


test_acc, macro_f1 = inference(test_dataset)
print(f"\nTest acc : {test_acc:.4f}")
print(f"Test macro-F1 : {macro_f1:.4f}")

  0%|          | 0/1923 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 26%|██▌       | 500/1923 [00:30<01:26, 16.48it/s]

              precision    recall  f1-score   support

           0       0.68      0.12      0.21       225
           1       0.51      0.57      0.54       204
           2       0.21      0.68      0.32        71

    accuracy                           0.38       500
   macro avg       0.47      0.46      0.36       500
weighted avg       0.54      0.38      0.36       500

[[ 28  90 107]
 [ 12 116  76]
 [  1  22  48]]

Test acc : 0.3840
Test macro-F1 : 0.3551


In [ ]:
prompt_type = 'zero_shot'

In [ ]:
templates[prompt_type][model_size].append(
    {
        'preamble': preamble,
        'options': options,
        'template': template,
        'ans_type': ans_type,
        'acc': test_acc,
        'macro_f1': macro_f1
    }
)

In [ ]:
from pprint import pprint
pprint(templates)

{'few_shot': {'base': [], 'small': []},
 'zero_shot': {'base': [{'acc': 0.5039001560062403,
                         'ans_type': 'label',
                         'macro_f1': 0.4232290037291178,
                         'options': '',
                         'preamble': 'Is the sentence normal, hatespeech or '
                                     'offensive? Classify the sentence into '
                                     'one of these types - normal, hatespeech, '
                                     'offensive. \n'
                                     'Sentence: ',
                         'template': '$preamble$prompt\n$options\nAnswer:'},
                        {'acc': 0.4674986999479979,
                         'ans_type': 'label',
                         'macro_f1': 0.4123361399509268,
                         'options': '',
                         'preamble': 'Classify the sentence into one of these '
                                     'types - normal, hatespeech, offens

In [ ]:
import json
with open('templates.json', 'r') as f:
  json.dump(templates, f)

# Few-shot Inference

In [ ]:
fewshot_prompts = [
    'Sentence: boatloads of refugee women children went missing in the mediterranean no explanations and how europe got flooded with nothing but middle east men at least those immigrants got to live others are not so lucky\nAnswer:normal',
    'Sentence: what your advice to a straight white conservative christian male with strong opinions trying to get a job\nAnswer:normal'
    'Sentence: either way kikes and sand niggers will most likely die this is very good\nAnswer:hatespeech',
    'Sentence: k nigger keep looking your daddy aint here try the white woman house down the corner\nAnswer:hatespeech',
    'Sentence: senile retarded democrat sounds about right\nAnswer:offensive',
    'Sentence: train guy as in the driver this is what happens when your base is mentally retarded and has mental issues\nAnswer:offensive',
]

fewshot = '\n\n'.join(fewshot_prompts)

In [ ]:
fewshot

'Sentence: boatloads of refugee women children went missing in the mediterranean no explanations and how europe got flooded with nothing but middle east men at least those immigrants got to live others are not so lucky\nAnswer:normal\n\nSentence: what your advice to a straight white conservative christian male with strong opinions trying to get a job\nAnswer:normalSentence: either way kikes and sand niggers will most likely die this is very good\nAnswer:hatespeech\n\nSentence: k nigger keep looking your daddy aint here try the white woman house down the corner\nAnswer:hatespeech\n\nSentence: senile retarded democrat sounds about right\nAnswer:offensive\n\nSentence: train guy as in the driver this is what happens when your base is mentally retarded and has mental issues\nAnswer:offensive'

In [ ]:
prompt_type = 'few_shot'
preamble = 'Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.'
options = ''
ans_type = 'label'
template = '$preamble\n$fewshot\n\nSentence: $prompt$options\nAnswer:'
test_dataset = HateSpeechDataset(test, tokenizer, prompt_type, fewshot, preamble, ans_type, options, Template(template), device)

In [ ]:
test_dataset[0]

Is the sentence normal, hatespeech or offensive? Classify the sentence into one of these types - normal, hatespeech, offensive.
Sentence: tariq nasheed beating the fuck out of that drum beloved lmao somebody put a biscuit in that niggas mouth already
Answer:normal

Sentence: <number> <number> <percent> of brits think multiculturalism is a failure i guess the other <percent> are happy with their muzzie master there no great in britain anymore soon big ben will be converted into a minaret
Answer:hatespeech

Sentence: i saw a couple baums and steins in the victim list but that about it ya got more info i thought the school was half spic though
Answer:offensive

Sentence: laura loomer raped me while screaming at me in her disgusting kike language and said we must exterminate the goyim laura loomer loomergate
Answer:


({'input_ids': tensor([[   27,     7,     8,  7142,  1389,     6,  5591,     7,   855, 10217,
             42, 12130,    58,  4501,  4921,     8,  7142,   139,    80,    13,
            175,  1308,     3,    18,  1389,     6,  5591,     7,   855, 10217,
              6, 12130,     5,  4892,    17,  1433,    10,     3,  5310,  1824,
              3,    29,     9,     7, 24778, 16201,     8,     3,    89,  4636,
             91,    13,    24,  5253, 11479,     3,    40,    51,     9,    32,
          10843,   474,     3,     9, 18922,    16,    24,     3,  7315,  5556,
           4247,   641, 11801,    10, 12110,  4892,    17,  1433,    10,     3,
              2,  5525,  1152,  3155,     3,     2,  5525,  1152,  3155,     3,
              2,   883,  3728,  3155,    13,     3,  2160,    17,     7,   317,
          29142,   159,    51,    19,     3,     9,  3338,     3,    23,  3382,
              8,   119,     3,     2,   883,  3728,  3155,    33,  1095,    28,
             70,  4035, 21

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm import tqdm

def inference(data):
  true_labels, pred_labels = [], []
  for i in tqdm(range(len(data))):
    input, label = data[i]
    output = model.generate(**input)
    pred = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    # print(pred)
    pred_label = post_process(pred, data.ans_type)
    true_labels.append(label)
    pred_labels.append(pred_label)

  assert len(true_labels) == len(pred_labels)
  num_correct = sum(1 for a, b in zip(pred_labels, true_labels) if a == b)
  test_acc = num_correct / len(true_labels)
  macro_f1 = f1_score(true_labels, pred_labels, average='macro')

  print(classification_report(true_labels, pred_labels))
  print(confusion_matrix(true_labels, pred_labels))

  return test_acc, macro_f1


test_acc, macro_f1 = inference(test_dataset)
print(f"\nTest acc : {test_acc:.4f}")
print(f"Test macro-F1 : {macro_f1:.4f}")

  0%|          | 0/1923 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1923/1923 [02:52<00:00, 11.12it/s]

              precision    recall  f1-score   support

           0       0.60      0.58      0.59       781
           1       0.46      0.79      0.58       594
           2       0.42      0.12      0.19       548

    accuracy                           0.51      1923
   macro avg       0.50      0.50      0.45      1923
weighted avg       0.51      0.51      0.47      1923

[[452 259  70]
 [105 468  21]
 [201 280  67]]

Test acc : 0.5133
Test macro-F1 : 0.4539


In [ ]:
templates[prompt_type][model_size].append(
    {
        'preamble': preamble,
        'fewshot': fewshot,
        'options': options,
        'template': template,
        'ans_type': ans_type,
        'acc': test_acc,
        'macro_f1': macro_f1
    }
)

In [ ]:
from pprint import pprint
pprint(templates)

{'few_shot': {'base': [{'acc': 0.608,
                        'ans_type': 'label',
                        'fewshot': 'Sentence: tariq nasheed beating the fuck '
                                   'out of that drum beloved lmao somebody put '
                                   'a biscuit in that niggas mouth already\n'
                                   'Answer:normal\n'
                                   '\n'
                                   'Sentence: <number> <number> <percent> of '
                                   'brits think multiculturalism is a failure '
                                   'i guess the other <percent> are happy with '
                                   'their muzzie master there no great in '
                                   'britain anymore soon big ben will be '
                                   'converted into a minaret\n'
                                   'Answer:hatespeech\n'
                                   '\n'
                                  